# 기본 설정

In [127]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS as stopwords
from PIL import Image, ImageFilter
from wordcloud import ImageColorGenerator
from matplotlib import font_manager, rc
from collections import Counter
import konlpy
from konlpy.tag import Mecab
import time
import seaborn as sns
import matplotlib.pyplot as plt
from random import shuffle
import json
from tqdm.notebook import tqdm
tqdm.pandas()
import gensim 
from gensim.models import Word2Vec, FastText, Doc2Vec
import fasttext
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc, show
from bokeh.transform import factor_cmap, factor_mark
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Loading BokehJS ...

In [128]:
bills = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/bills_24839_all_cleansed.csv')
senators = pd.read_csv('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/(원본) assembly_memebers_295.csv')

In [129]:
senators['한글이름'] = senators['이름'].str.split(' ').str[0]
senators['한자이름'] = senators['이름'].str.split(' ').str[1]

In [130]:
bills = bills[bills['발의자'] != '위원장']
bills = bills[bills['발의자'] != '정부'] 
bills = bills.dropna()

# 유저 인풋 만들기

In [131]:
embedding_model = Word2Vec.load('../models/word2vec ver21.0.model')

In [132]:
with open('/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학 이후/코딩 관련/개인프로젝트_국회의원 추천 (20대)/1차 데이터 및 jupyter/senators ver21.0.json', 'r') as f:
    senators_json = json.load(f)

In [133]:
word_dict = {}
for vocab, vector in zip(embedding_model.wv.index2word, embedding_model.wv.vectors):
    word_dict[vocab] = vector

### 함수 모음

In [134]:
def search(user_input):
    temp_list = []
    for word in word_dict:
        if user_input in word:
            temp_list.append(word)
    return temp_list

def similarWord(model, text):
    return model.most_similar(positive=[text], topn=30)

def similar_senators(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(round(similarity[i]*100, 2)) + "%"
    return result

def similar_senators_all(user_input, senators_json, word2vec_word_dict):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # senator 찾기
    similarity = {}
    for item in senators_json:
        sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
        similarity['{} {} {}'.format(senators_json[item]['한자이름'], senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    result = {}
    for i in list(rating):
        result[i] = round(similarity[i]*100, 2)
    return result


def similar_senators_with_q_all(user_input, senators_json, senators_df, word2vec_word_dict, quartile):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # quartile 안에 있는 senator 구하기
    list_senators = senators_df[senators_df['제정및전부개정법률안 비율'] > senators_df['제정및전부개정법률안 비율'].quantile(q=quartile, interpolation='nearest')]['이름'].to_list()
    # senator 찾기
    similarity = {}
    for item in senators_json:
        if item in list_senators:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
            similarity['{} {} {}'.format(senators_json[item]['한자이름'], senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
        else:
            continue
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    result = {}
    for i in list(rating):
        result[i] = round(similarity[i]*100, 2)
    return result

def similar_senators_with_q(user_input, senators_json, senators_df, word2vec_word_dict, quartile):
    # user_vector 만들기
    list_vector =[]
    for word in user_input:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = np.sum(list_vector, axis=0).tolist()
    
    # quartile 안에 있는 senator 구하기
    list_senators = senators_df[senators_df['제정및전부개정법률안 비율'] > senators_df['제정및전부개정법률안 비율'].quantile(q=quartile, interpolation='nearest')]['이름'].to_list()
    # senator 찾기
    similarity = {}
    for item in senators_json:
        if item in list_senators:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array(senators_json[item]['vector']).reshape(1,-1))
            similarity['{} {}'.format(senators_json[item]['한글이름'], senators_json[item]['정당'])] = float(sim)
        else:
            continue
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(round(similarity[i]*100, 2)) + "%"
    return result

def show_senators_bills_contents(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_ids = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['bill_id'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_ids.extend(semi_result)
        list_temp_keyword_bills_ids = list(set(list_temp_keyword_bills_ids))
        shuffle(list_temp_keyword_bills_ids)
        list_bills_title = df_bills[df_bills['bill_id'].isin(list_temp_keyword_bills_ids[:3])]['법안명'].values.tolist()
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

def show_senators_bills_titles(senators_list, df_bills):
    dict_senator_bills = {}
    for senator in senators_list:
        senator_name = senator.split(" ", 1)[0]
        bills_id_bySenator = df_bills[df_bills['발의자'].str.contains(senator_name)].bill_id.values.tolist()

        list_temp_keyword_bills_titles = []
        for keyword in user_input:
            bills_id_byKeyword = df_bills.loc[bills_id_bySenator]['bill_id'].values.tolist()
            df_semi = df_bills.loc[bills_id_byKeyword]
            semi_result = df_semi[df_semi['법안명_법안내용'].str.contains(keyword)]['법안명'].drop_duplicates().values.tolist()
            list_temp_keyword_bills_titles.extend(semi_result)
        list_temp_keyword_bills_titles = list(set(list_temp_keyword_bills_titles))
        shuffle(list_temp_keyword_bills_titles)
        list_bills_titlle = df_bills.loc(list_temp_keyword_bills_titles[:3])
        dict_senator_bills[senator] = list_bills_title
    return dict_senator_bills

In [135]:
def count_bills(name, bills_df):
    df_temp = bills_df[bills_df['발의자'].str.contains(name)]
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_

def count_bills_all(bills_df):
    df_temp = bills_df
    bills_deleted = df_temp[(df_temp['상태']=='폐기') | (df_temp['상태']=='철회')].shape[0]
    df_temp_save = df_temp[~(df_temp['상태']=='폐기') & ~(df_temp['상태']=='철회')]
    bills_일부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('일부개정법률안')].shape[0]
    bills_전부개정법률안 = df_temp_save[df_temp_save['법안명'].str.contains('전부개정법률안')].shape[0]
    bills_제정법안 = df_temp_save[(~df_temp_save['법안명'].str.contains('일부개정법률안'))& (~df_temp_save['법안명'].str.contains('전부개정법률안'))].shape[0]
    dict_ = {'일부개정법률안 비율': str(round((bills_일부개정법률안/df_temp.shape[0])*100,2))+'%',
            '전부개정법률안 비율': str(round((bills_전부개정법률안/df_temp.shape[0])*100,2))+'%',
            '제정법률안 비율': str(round((bills_제정법안/df_temp.shape[0])*100,2))+"%",
            '폐기법률안 비율': str(round((bills_deleted/df_temp.shape[0])*100,2))+'%'}
    return dict_


def get_quantile(senator):
    result = result
    return result

# 전체 distribution 분포 살펴보기

In [136]:
dict_bills_portion = {}
for senator in senators['한글이름']:
    dict_bills_portion[senator] = count_bills(senator, bills)

In [137]:
df_bills_portion = pd.DataFrame(dict_bills_portion).T

In [138]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].str[:-1]
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].str[:-1]
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].str[:-1]
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].str[:-1]

In [139]:
df_bills_portion['일부개정법률안 비율'] = df_bills_portion['일부개정법률안 비율'].astype(float)
df_bills_portion['전부개정법률안 비율'] = df_bills_portion['전부개정법률안 비율'].astype(float)
df_bills_portion['폐기법률안 비율'] = df_bills_portion['폐기법률안 비율'].astype(float)
df_bills_portion['제정법률안 비율'] = df_bills_portion['제정법률안 비율'].astype(float)

In [140]:
df_bills_portion['제정및전부개정법률안 비율'] = df_bills_portion['제정법률안 비율'] + df_bills_portion['전부개정법률안 비율']

# 상위 % 잘라낸 뒤 추천하기

In [141]:
df_bills_portion.reset_index(inplace=True)

In [142]:
senators = senators.merge(df_bills_portion, left_on="한글이름", right_on="index", how="left")
senators = senators.drop(columns=['index'])

In [143]:
df_temp = senators[['이름', '한글이름','제정및전부개정법률안 비율']].sort_values(['제정및전부개정법률안 비율'])

In [144]:
df_temp['순위'] = range(1, df_temp.shape[0]+1)

In [145]:
df_temp['percentile'] = round(100-(df_temp['순위']/295) * 100, 5)

In [146]:
df_temp['percentile'] = df_temp['percentile'].astype(str)
df_temp['percentile'] = df_temp['percentile'] + '%'
df_temp

,이름,한글이름,제정및전부개정법률안 비율,순위,percentile
284,허윤정 許允貞,허윤정,0.00,1,99.66102%
245,정은혜 鄭恩惠,정은혜,0.00,2,99.32203%
248,정점식 鄭点植,정점식,2.63,3,98.98305%
271,최도자 崔道子,최도자,3.89,4,98.64407%
224,임재훈 林哉勳,임재훈,4.32,5,98.30508%
14,권칠승 權七勝,권칠승,4.38,6,97.9661%
181,이규희 李揆熙,이규희,4.41,7,97.62712%
220,이후삼 李厚三,이후삼,4.42,8,97.28814%
15,금태섭 琴泰燮,금태섭,4.44,9,96.94915%
136,신용현 申容賢,신용현,4.49,10,96.61017%


# 의원별 입법지수 계산

In [147]:
senators

,이름,정당,선거구,당선횟수,소속위원회,학력,주요경력,연락처,이메일,대표발의법안수,위원회 출석률,본회의 출석률,재산,한글이름,한자이름,일부개정법률안 비율,전부개정법률안 비율,제정법률안 비율,폐기법률안 비율,제정및전부개정법률안 비율
0,강길부 姜吉夫,무소속,울산 울주군,"4선 [ 17대, 18대, 19대, 20대 ]",산업통상자원중소벤처기업위원회,언양농업고등학교/성균관대학교 행정학과/서울대학교환경대학원 도시계획학 석사,6대 건설교통부 차관/한국감정원 원장,의원회관 606호 | 전화 02-784-5073 | 팩스 02-788-0103,shinmut@assembly.go.kr,2,93.65 %,82.88 %,"총계 3,386,959 3,436,703 3,394,464 3,762,663",강길부,姜吉夫,92.23,0.32,6.47,0.97,6.79
1,강병원 姜炳遠,더불어민주당,서울 은평구을,1선,기획재정위원회,서울대학교 농업생명과학대학 농경제학과,서울대학교 총학생회장/노무현대통령 청와대 행정관,의원회관 610호 | 전화 02-784-1422 | 팩스 02-788-0118,kangbw89@gmail.com,14,98.15 %,98.63 %,"총계 662,157 754,761 888,275 989,750",강병원,姜炳遠,89.65,1.34,7.48,1.53,8.82
2,강석진 姜錫振,미래통합당,경남 산청군함양군거창군합천군,1선,농림축산식품해양수산위원회,연세대학교 행정대학원,새누리당 최경환 원내대표 비서실장/경남 거창군수,의원회관 542호 | 전화 02-784-1460 | 팩스 02-788-0119,ksjv7900@naver.com,19,98.36 %,93.84 %,"총계 365,671 472,584 532,440 724,182",강석진,姜錫振,88.97,0.15,8.38,2.50,8.53
3,강석호 姜碩鎬,미래통합당,경북 영양군영덕군봉화군울진군,"3선 [ 18대, 19대, 20대 ]",농림축산식품해양수산위원회,한국외국어대학교 서양어대학 스페인어과,새누리당 제1사무부총장/경상북도의회 의원/스텔베르그&삼일 대표이사,의원회관 707호 | 전화 02-784-2374 | 팩스 02-788-0105,kangsh@na.go.kr,8,88.46 %,86.99 %,"총계 15,371,209 13,904,477 16,757,244 16,732,344",강석호,姜碩鎬,87.46,0.35,10.28,1.92,10.63
4,강창일 姜昌一,더불어민주당,제주 제주시갑,"4선 [ 17대, 18대, 19대, 20대 ]",행정안전위원회,서울대학교 국사학과/도쿄대학교대학원 문학 박사,5.18기념재단 이사/배재대학교 교무처장,의원회관 806호 | 전화 02-784-6084 | 팩스 02-788-0108,kangci@assembly.go.kr,10,83.33 %,90.41 %,"총계 1,618,382 2,111,355 2,257,761 1,866,353",강창일,姜昌一,90.48,0.82,5.95,2.74,6.77
5,강효상 姜孝祥,미래통합당,비례대표,1선,공공부문 채용비리 의혹과 관련된 국정조사특별위원회,서울대학교 법과대학 사법학과 졸업,TV조선 보도본부장/조선일보 편집국장,의원회관 418호 | 전화 02-784-6195 | 팩스 02-788-0400,hyoshang@gmail.com,12,88.89 %,83.56 %,"총계 2,696,829 2,824,614 3,096,682 3,468,624",강효상,姜孝祥,88.02,0.00,9.38,2.60,9.38
6,강훈식 姜勳植,더불어민주당,충남 아산시을,1선,국토교통위원회,건국대학교 경영정보학과,더불어민주당 아산을 지역위원장/동국대 언론정보대학원 겸임교수,의원회관 640호 | 전화 02-784-1045 | 팩스 02-788-0263,gohoonsik@gmail.com,7,83.33 %,93.84 %,"총계 333,560 209,253 298,004 506,666",강훈식,姜勳植,91.07,0.62,7.20,1.11,7.82
7,경대수 慶大秀,미래통합당,충북 증평군진천군음성군,"2선 [ 19대, 20대 ]",농림축산식품해양수산위원회,경동고등학교/서울대학교/서울대학교 대학원 법학과 석사 수료,새누리당 충북도당 위원장/대검찰청 마약조직범죄부장 검사장/제주지방검찰청 검사장/제2...,의원회관 941호 | 전화 02-784-3977 | 팩스 02-788-0110,kyungds@na.go.kr,7,98.36 %,93.84 %,"총계 681,739 949,726 751,248 862,403",경대수,慶大秀,91.54,0.33,6.72,1.41,7.05
8,고용진 高榕禛,더불어민주당,서울 노원구갑,1선,국회운영위원회,서울대학교대학원 언론정보학과 석사,서울시의회의원/노무현정부 청와대 행정관/더불어민주당 노원갑 지역위원장,의원회관 1005호 | 전화 02-784-4840 | 팩스 02-788-0171,gogoyongjin@naver.com,4,100.00 %,95.89 %,"총계 632,899 875,972 1,184,064 1,576,501",고용진,高榕禛,92.33,0.79,4.91,1.98,5.70
9,곽대훈 郭大勳,미래통합당,대구 달서구갑,1선,산업통상자원중소벤처기업위원회,서울대학교 행정대학원 행정학 석사 수료,대구광역시 달서구청장/대구광역시 행정관리국장,의원회관 530호 | 전화 02-784-0790 | 팩스 02-788-0331,dhkwak605@naver.com,4,86.57 %,91.10 %,"총계 1,620,248 1,701,690 1,741,160 1,822,481",곽대훈,郭大勳,92.10,0.39,6.45,1.06,6.84


In [148]:
def count_senators(data):
    data = [x for x in data if '/' in x]
    return len(data)

In [149]:
dict_count_biils = {}
for row in senators.itertuples():
    dict_temp = {}
    dict_temp['한글이름'] = row[-7]
    
    # Get the dataframe
    df_temp = bills[bills['발의자'].str.contains(row[-7])]
    bills_전부개정법률안 = df_temp[df_temp['법안명'].str.contains('전부개정법률안')]
    bills_제정법안 = df_temp[(~df_temp['법안명'].str.contains('일부개정법률안'))& (~df_temp['법안명'].str.contains('전부개정법률안'))]
    bills_total = pd.concat([bills_제정법안, bills_전부개정법률안])
    
    # count mean senators proposing 
    bills_total['발의자 수'] = bills_total['발의자'].str.split(' ').map(count_senators)
    
    # save in dictionary
    dict_temp['전체 발의법안 수'] = bills_total.shape[0]
    if dict_temp['전체 발의법안 수'] == 0:
        dict_temp['평균 발의자 수'] = 0.000000000000009
    else:    
        dict_temp['평균 발의자 수'] = round(bills_total['발의자 수'].mean())

    dict_count_biils[row[-6]] = dict_temp

In [150]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [151]:
df_result = pd.DataFrame(dict_count_biils).T.reset_index().rename(columns={'index':'한자이름'})
df_result['법안수/평균 발의자 수'] = df_result['전체 발의법안 수'] / df_result['평균 발의자 수'] 
df_result.sort_values(by='법안수/평균 발의자 수')
df_result['bills_index'] = df_result['법안수/평균 발의자 수'].astype(float)
df_result['bills_index_scaled'] = scaler.fit_transform(df_result[['bills_index']])
df_result

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,姜吉夫,강길부,23,67,0.343284,0.343284,0.031618
1,姜炳遠,강병원,94,38,2.47368,2.473684,0.227839
2,姜錫振,강석진,61,47,1.29787,1.297872,0.119541
3,姜碩鎬,강석호,63,47,1.34043,1.340426,0.123460
4,姜昌一,강창일,78,38,2.05263,2.052632,0.189058
5,姜孝祥,강효상,38,63,0.603175,0.603175,0.055556
6,姜勳植,강훈식,116,34,3.41176,3.411765,0.314241
7,慶大秀,경대수,67,43,1.55814,1.558140,0.143513
8,高榕禛,고용진,75,34,2.20588,2.205882,0.203173
9,郭大勳,곽대훈,74,46,1.6087,1.608696,0.148169


In [152]:
bills_index = df_result.copy()
bills_index

,한자이름,한글이름,전체 발의법안 수,평균 발의자 수,법안수/평균 발의자 수,bills_index,bills_index_scaled
0,姜吉夫,강길부,23,67,0.343284,0.343284,0.031618
1,姜炳遠,강병원,94,38,2.47368,2.473684,0.227839
2,姜錫振,강석진,61,47,1.29787,1.297872,0.119541
3,姜碩鎬,강석호,63,47,1.34043,1.340426,0.123460
4,姜昌一,강창일,78,38,2.05263,2.052632,0.189058
5,姜孝祥,강효상,38,63,0.603175,0.603175,0.055556
6,姜勳植,강훈식,116,34,3.41176,3.411765,0.314241
7,慶大秀,경대수,67,43,1.55814,1.558140,0.143513
8,高榕禛,고용진,75,34,2.20588,2.205882,0.203173
9,郭大勳,곽대훈,74,46,1.6087,1.608696,0.148169


# 정규화하고 scaling (예시 1)

In [153]:
user_input=['복지서비스','장애인', '특수교육','장애인복지시설']

In [154]:
dict_similarity = similar_senators_all(user_input, senators_json ,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_index_scaled']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_index_scaled']
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled','quality_index_time', 'quality_index_sum']]

In [155]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
75,李燦烈 이찬열 미래통합당,52.80,0.602958,1.000000,0.602958,1.602958
56,黃柱洪 황주홍 민생당,53.05,0.631399,0.833333,0.526166,1.464733
20,金相姬 김상희 더불어민주당,54.15,0.756542,0.448642,0.339416,1.205183
137,申昌賢 신창현 더불어민주당,52.23,0.538111,0.586404,0.315550,1.124515
193,朴釘 박정 더불어민주당,51.69,0.476678,0.651089,0.310360,1.127767
190,金鍾懷 김종회 민생당,51.72,0.480091,0.632723,0.303765,1.112814
96,盧雄來 노웅래 더불어민주당,52.60,0.580205,0.487267,0.282714,1.067471
41,蘇秉勳 소병훈 더불어민주당,53.40,0.671217,0.420230,0.282066,1.091448
38,鄭仁和 정인화 무소속,53.44,0.675768,0.414474,0.280088,1.090242
12,奇東旻 기동민 더불어민주당,54.23,0.765643,0.362839,0.277805,1.128482


In [156]:
df_merged.sort_values(by=['quality_index_sum'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
75,李燦烈 이찬열 미래통합당,52.80,0.602958,1.000000,0.602958,1.602958
56,黃柱洪 황주홍 민생당,53.05,0.631399,0.833333,0.526166,1.464733
20,金相姬 김상희 더불어민주당,54.15,0.756542,0.448642,0.339416,1.205183
12,奇東旻 기동민 더불어민주당,54.23,0.765643,0.362839,0.277805,1.128482
193,朴釘 박정 더불어민주당,51.69,0.476678,0.651089,0.310360,1.127767
137,申昌賢 신창현 더불어민주당,52.23,0.538111,0.586404,0.315550,1.124515
16,尹昭夏 윤소하 정의당,54.16,0.757679,0.356908,0.270422,1.114587
190,金鍾懷 김종회 민생당,51.72,0.480091,0.632723,0.303765,1.112814
41,蘇秉勳 소병훈 더불어민주당,53.40,0.671217,0.420230,0.282066,1.091448
38,鄭仁和 정인화 무소속,53.44,0.675768,0.414474,0.280088,1.090242


In [157]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
0,余永國 여영국 정의당,56.29,1.000000,0.072368,0.072368,1.072368
1,吳怜勳 오영훈 더불어민주당,55.07,0.861206,0.175000,0.150711,1.036206
2,鄭春淑 정춘숙 더불어민주당,54.86,0.837315,0.250731,0.209941,1.088046
3,劉承旼 유승민 미래통합당,54.53,0.799772,0.144737,0.115757,0.944509
4,羅卿瑗 나경원 미래통합당,54.43,0.788396,0.053614,0.042269,0.842009
5,金世淵 김세연 미래통합당,54.41,0.786121,0.122807,0.096541,0.908928
6,全希卿 전희경 미래통합당,54.39,0.783845,0.066697,0.052280,0.850542
7,朴炅美 박경미 더불어민주당,54.35,0.779295,0.154904,0.120716,0.934199
8,都鐘煥 도종환 더불어민주당,54.29,0.772469,0.080297,0.062027,0.852766
9,南仁順 남인순 더불어민주당,54.29,0.772469,0.224845,0.173686,0.997314


# 예시 2

In [158]:
user_input=['취업', '비정규직', '근로조건','청년실업률','취업지원']

In [159]:
dict_similarity = similar_senators_with_q_all(user_input, senators_json, senators ,word_dict, 0.55)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_index_scaled']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_index_scaled']
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled','quality_index_time', 'quality_index_sum']]

In [160]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
22,金相姬 김상희 더불어민주당,54.94,0.748843,0.448642,0.335962,1.197484
66,盧雄來 노웅래 더불어민주당,53.74,0.609954,0.487267,0.297210,1.097220
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
56,朴洪根 박홍근 더불어민주당,53.99,0.638889,0.407046,0.260057,1.045935
67,魏聖坤 위성곤 더불어민주당,53.73,0.608796,0.425987,0.259339,1.034783
90,柳成葉 유성엽 민생당,52.97,0.520833,0.447368,0.233004,0.968202
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
27,朴柱民 박주민 더불어민주당,54.62,0.711806,0.311969,0.222062,1.023775
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392


In [161]:
df_merged.sort_values(by=['quality_index_sum'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
22,金相姬 김상희 더불어민주당,54.94,0.748843,0.448642,0.335962,1.197484
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392
66,盧雄來 노웅래 더불어민주당,53.74,0.609954,0.487267,0.297210,1.097220
2,金富謙 김부겸 더불어민주당,57.08,0.996528,0.073266,0.073011,1.069793
11,權美赫 권미혁 더불어민주당,55.64,0.829861,0.233997,0.194185,1.063858
8,金鍾勳 김종훈 민중당,56.25,0.900463,0.157470,0.141796,1.057933
56,朴洪根 박홍근 더불어민주당,53.99,0.638889,0.407046,0.260057,1.045935


In [162]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
2,金富謙 김부겸 더불어민주당,57.08,0.996528,0.073266,0.073011,1.069793
3,金太年 김태년 더불어민주당,56.71,0.953704,0.074045,0.070617,1.027749
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
5,洪永杓 홍영표 더불어민주당,56.36,0.913194,0.094024,0.085862,1.007219
6,余永國 여영국 정의당,56.34,0.910880,0.072368,0.065919,0.983248
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392
8,金鍾勳 김종훈 민중당,56.25,0.900463,0.157470,0.141796,1.057933
9,吳濟世 오제세 더불어민주당,56.04,0.876157,0.166301,0.145706,1.042459


# 유사도 상위 20% 국회의원들만 가지고 하기

### 예시 1

In [163]:
user_input=['복지서비스','장애인', '특수교육','장애인복지시설']

In [164]:
dict_similarity = similar_senators_with_q_all(user_input, senators_json, senators ,word_dict, 0.55)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])

df_similarity_top = df_similarity[df_similarity['similarity'] >= df_similarity['similarity'].quantile(q=0.8, interpolation='nearest')]

df_merged = df_similarity_top.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_index_scaled']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_index_scaled']
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled','quality_index_time', 'quality_index_sum']]

In [165]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
12,金相姬 김상희 더불어민주당,54.15,0.743097,0.448642,0.333384,1.191739
18,朴柱民 박주민 더불어민주당,53.50,0.665066,0.311969,0.207480,0.977035
15,權美赫 권미혁 더불어민주당,54.05,0.731092,0.233997,0.171074,0.965090
11,兪銀惠 유은혜 더불어민주당,54.16,0.744298,0.226529,0.168605,0.970827
20,金映豪 김영호 더불어민주당,53.45,0.659064,0.252632,0.166500,0.911695
1,吳怜勳 오영훈 더불어민주당,55.07,0.853541,0.175000,0.149370,1.028541
8,安敏錫 안민석 더불어민주당,54.20,0.749100,0.193906,0.145255,0.943005
16,尹種畢 윤종필 미래통합당,53.80,0.701080,0.193906,0.135944,0.894986
23,李種明 이종명 미래한국당,53.18,0.626651,0.214912,0.134675,0.841563
13,吳濟世 오제세 더불어민주당,54.10,0.737095,0.166301,0.122580,0.903396


In [166]:
df_merged.sort_values(by=['quality_index_sum'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
12,金相姬 김상희 더불어민주당,54.15,0.743097,0.448642,0.333384,1.191739
0,余永國 여영국 정의당,56.29,1.000000,0.072368,0.072368,1.072368
1,吳怜勳 오영훈 더불어민주당,55.07,0.853541,0.175000,0.149370,1.028541
18,朴柱民 박주민 더불어민주당,53.50,0.665066,0.311969,0.207480,0.977035
11,兪銀惠 유은혜 더불어민주당,54.16,0.744298,0.226529,0.168605,0.970827
15,權美赫 권미혁 더불어민주당,54.05,0.731092,0.233997,0.171074,0.965090
8,安敏錫 안민석 더불어민주당,54.20,0.749100,0.193906,0.145255,0.943005
2,劉承旼 유승민 미래통합당,54.53,0.788715,0.144737,0.114156,0.933452
6,朴炅美 박경미 더불어민주당,54.35,0.767107,0.154904,0.118828,0.922011
20,金映豪 김영호 더불어민주당,53.45,0.659064,0.252632,0.166500,0.911695


In [167]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
0,余永國 여영국 정의당,56.29,1.000000,0.072368,0.072368,1.072368
1,吳怜勳 오영훈 더불어민주당,55.07,0.853541,0.175000,0.149370,1.028541
2,劉承旼 유승민 미래통합당,54.53,0.788715,0.144737,0.114156,0.933452
3,羅卿瑗 나경원 미래통합당,54.43,0.776711,0.053614,0.041642,0.830324
4,金世淵 김세연 미래통합당,54.41,0.774310,0.122807,0.095091,0.897117
5,全希卿 전희경 미래통합당,54.39,0.771909,0.066697,0.051484,0.838606
6,朴炅美 박경미 더불어민주당,54.35,0.767107,0.154904,0.118828,0.922011
7,申相珍 신상진 미래통합당,54.26,0.756303,0.084026,0.063549,0.840328
8,安敏錫 안민석 더불어민주당,54.20,0.749100,0.193906,0.145255,0.943005
9,朴順子 박순자 미래통합당,54.17,0.745498,0.076225,0.056826,0.821723


### 예시 2

In [168]:
user_input=['취업', '비정규직', '근로조건','청년실업률','취업지원']

In [169]:
dict_similarity = similar_senators_with_q_all(user_input, senators_json, senators ,word_dict, 0.55)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])

df_similarity_top = df_similarity[df_similarity['similarity'] >= df_similarity['similarity'].quantile(q=0.8, interpolation='nearest')]

df_merged = df_similarity_top.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_index_scaled']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_index_scaled']
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled','quality_index_time', 'quality_index_sum']]

In [170]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
22,金相姬 김상희 더불어민주당,54.94,0.748843,0.448642,0.335962,1.197484
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392
21,金鍾民 김종민 더불어민주당,54.95,0.750000,0.281579,0.211184,1.031579
11,權美赫 권미혁 더불어민주당,55.64,0.829861,0.233997,0.194185,1.063858
18,閔丙梪 민병두 더불어민주당,55.06,0.762731,0.247368,0.188676,1.010100
26,李龍得 이용득 더불어민주당,54.73,0.724537,0.255848,0.185371,0.980385
14,兪銀惠 유은혜 더불어민주당,55.45,0.807870,0.226529,0.183006,1.034400


In [171]:
df_merged.sort_values(by=['quality_index_sum'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
22,金相姬 김상희 더불어민주당,54.94,0.748843,0.448642,0.335962,1.197484
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392
2,金富謙 김부겸 더불어민주당,57.08,0.996528,0.073266,0.073011,1.069793
11,權美赫 권미혁 더불어민주당,55.64,0.829861,0.233997,0.194185,1.063858
8,金鍾勳 김종훈 민중당,56.25,0.900463,0.157470,0.141796,1.057933
9,吳濟世 오제세 더불어민주당,56.04,0.876157,0.166301,0.145706,1.042459
14,兪銀惠 유은혜 더불어민주당,55.45,0.807870,0.226529,0.183006,1.034400


In [172]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,quality_index_time,quality_index_sum
0,沈相奵 심상정 정의당,57.11,1.000000,0.228675,0.228675,1.228675
1,李貞味 이정미 정의당,57.09,0.997685,0.289474,0.288804,1.287159
2,金富謙 김부겸 더불어민주당,57.08,0.996528,0.073266,0.073011,1.069793
3,金太年 김태년 더불어민주당,56.71,0.953704,0.074045,0.070617,1.027749
4,金鍾大 김종대 정의당,56.42,0.920139,0.249025,0.229138,1.169164
5,洪永杓 홍영표 더불어민주당,56.36,0.913194,0.094024,0.085862,1.007219
6,余永國 여영국 정의당,56.34,0.910880,0.072368,0.065919,0.983248
7,秋惠仙 추혜선 정의당,56.27,0.902778,0.245614,0.221735,1.148392
8,金鍾勳 김종훈 민중당,56.25,0.900463,0.157470,0.141796,1.057933
9,吳濟世 오제세 더불어민주당,56.04,0.876157,0.166301,0.145706,1.042459


# Change index to percentage

### 예시 1

In [173]:
user_input=['취업', '비정규직', '근로조건','청년실업률','취업지원']

In [174]:
dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

df_merged['rank'] = range(1,df_merged.shape[0]+1)

df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

In [175]:
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_quantile']

In [176]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
0,金賢美 김현미 더불어민주당,58.10,1.000000,0.076087,225,0.237288,0.237288,1.237288
1,沈相奵 심상정 정의당,57.11,0.922170,0.228675,76,0.742373,0.684594,1.664543
2,李貞味 이정미 정의당,57.09,0.920597,0.289474,40,0.864407,0.795771,1.785004
3,金富謙 김부겸 더불어민주당,57.08,0.919811,0.073266,233,0.210169,0.193316,1.129981
4,金太年 김태년 더불어민주당,56.71,0.890723,0.074045,231,0.216949,0.193242,1.107672
5,李彦周 이언주 미래통합당,56.55,0.878145,0.154737,135,0.542373,0.476282,1.420518
6,金鍾大 김종대 정의당,56.42,0.867925,0.249025,58,0.803390,0.697282,1.671314
7,洪永杓 홍영표 더불어민주당,56.36,0.863208,0.094024,196,0.335593,0.289687,1.198801
8,余永國 여영국 정의당,56.34,0.861635,0.072368,235,0.203390,0.175248,1.065025
9,李勇周 이용주 무소속,56.31,0.859277,0.250000,57,0.806780,0.693247,1.666056


In [177]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
2,李貞味 이정미 정의당,57.09,0.920597,0.289474,40,0.864407,0.795771,1.785004
10,尹昭夏 윤소하 정의당,56.29,0.857704,0.356908,28,0.905085,0.776295,1.762789
19,鄭東泳 정동영 민생당,55.81,0.819969,0.378655,20,0.932203,0.764377,1.752172
38,李燦烈 이찬열 미래통합당,55.13,0.766509,1.000000,1,0.996610,0.763911,1.763120
16,張貞淑 장정숙 민생당,55.95,0.830975,0.364879,26,0.911864,0.757736,1.742839
45,金相姬 김상희 더불어민주당,54.94,0.751572,0.448642,10,0.966102,0.726095,1.717674
29,金海永 김해영 더불어민주당,55.35,0.783805,0.368421,24,0.918644,0.720038,1.702449
57,鄭仁和 정인화 무소속,54.73,0.735063,0.414474,15,0.949153,0.697687,1.684215
6,金鍾大 김종대 정의당,56.42,0.867925,0.249025,58,0.803390,0.697282,1.671314
34,金三和 김삼화 미래통합당,55.23,0.774371,0.331579,30,0.898305,0.695621,1.672676


### 예시 2

In [178]:
user_input=['복지서비스','장애인', '특수교육','장애인복지시설']

In [179]:
dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

df_merged['rank'] = range(1,df_merged.shape[0]+1)

df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

In [180]:
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_quantile']

In [181]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
0,余永國 여영국 정의당,56.29,1.000000,0.072368,235,0.203390,0.203390,1.203390
1,吳怜勳 오영훈 더불어민주당,55.07,0.861206,0.175000,114,0.613559,0.528401,1.474765
2,鄭春淑 정춘숙 더불어민주당,54.86,0.837315,0.250731,56,0.810169,0.678367,1.647485
3,劉承旼 유승민 미래통합당,54.53,0.799772,0.144737,141,0.522034,0.417508,1.321806
4,羅卿瑗 나경원 미래통합당,54.43,0.788396,0.053614,259,0.122034,0.096211,0.910430
5,金世淵 김세연 미래통합당,54.41,0.786121,0.122807,166,0.437288,0.343761,1.223409
6,全希卿 전희경 미래통합당,54.39,0.783845,0.066697,241,0.183051,0.143484,0.966896
7,朴炅美 박경미 더불어민주당,54.35,0.779295,0.154904,134,0.545763,0.425310,1.325057
8,都鐘煥 도종환 더불어민주당,54.29,0.772469,0.080297,218,0.261017,0.201627,1.033486
9,南仁順 남인순 더불어민주당,54.29,0.772469,0.224845,80,0.728814,0.562986,1.501282


In [182]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
20,金相姬 김상희 더불어민주당,54.15,0.756542,0.448642,10,0.966102,0.730896,1.722643
12,奇東旻 기동민 더불어민주당,54.23,0.765643,0.362839,27,0.908475,0.695567,1.674117
16,尹昭夏 윤소하 정의당,54.16,0.757679,0.356908,28,0.905085,0.685764,1.662764
2,鄭春淑 정춘숙 더불어민주당,54.86,0.837315,0.250731,56,0.810169,0.678367,1.647485
21,印在謹 인재근 더불어민주당,54.14,0.755404,0.287480,41,0.861017,0.650416,1.616421
38,鄭仁和 정인화 무소속,53.44,0.675768,0.414474,15,0.949153,0.641407,1.624920
41,蘇秉勳 소병훈 더불어민주당,53.40,0.671217,0.420230,14,0.952542,0.639363,1.623760
56,黃柱洪 황주홍 민생당,53.05,0.631399,0.833333,2,0.993220,0.627119,1.624620
40,張貞淑 장정숙 민생당,53.41,0.672355,0.364879,26,0.911864,0.613097,1.584219
19,全惠淑 전혜숙 더불어민주당,54.16,0.757679,0.247368,59,0.800000,0.606143,1.557679


In [183]:
senators_list = ['김상희 더불어민주당', '기동민 더불어민주당', '정춘숙 더불어민주당', '안재근 더불어민주당', '윤소하 정의당']
show_senators_bills_contents(senators_list, bills)

{'김상희 더불어민주당': ['성폭력범죄의 처벌 등에 관한 특례법 일부개정법률안(신창현의원 등 10인)',
  '공직선거법 일부개정법률안(김부겸의원 등 16인)',
  '기초연금법 일부개정법률안(권미혁의원 등 28인)'],
 '기동민 더불어민주당': ['장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(진선미의원 등 19인)',
  '공공단체등 위탁선거에 관한 법률 일부개정법률안(인재근의원 등 14인)',
  '중증장애인생산품 우선구매 특별법 일부개정법률안(기동민의원 등 12인)'],
 '정춘숙 더불어민주당': ['화재예방, 소방시설 설치·유지 및 안전관리에 관한 법률 일부개정법률안(윤소하의원 등 10인)',
  '의료기기법 일부개정법률안(양승조의원 등 10인)',
  '국민영양관리법 일부개정법률안(남인순의원 등 10인)'],
 '안재근 더불어민주당': [],
 '윤소하 정의당': ['장애인활동 지원에 관한 법률 일부개정법률안(정춘숙의원 등 11인)',
  '장애인차별금지 및 권리구제 등에 관한 법률 일부개정법률안(김상희의원 등 14인)',
  '영유아보육법 일부개정법률안(권미혁의원 등 29인)']}

### 예시 3

In [235]:
user_input=['창업','청년','벤처']

In [236]:
dict_similarity = similar_senators_all(user_input, senators_json,word_dict)
df_similarity = pd.DataFrame(dict_similarity, index=['similarity']).T

df_similarity = df_similarity.reset_index().rename(columns={'index':'name_party'})
df_similarity['한자이름'] = df_similarity['name_party'].str.split(' ').str[0]
df_similarity['similarity_scaled'] = scaler.fit_transform(df_similarity[['similarity']])
df_merged = df_similarity.merge(bills_index[['한자이름', 'bills_index_scaled']], on = '한자이름',how='left')
df_merged = df_merged[['name_party', 'similarity', 'similarity_scaled', 'bills_index_scaled']]

df_merged = df_merged.sort_values(by=['bills_index_scaled'], ascending = False)

df_merged['rank'] = range(1,df_merged.shape[0]+1)

df_merged['bills_quantile'] = 1 - (df_merged['rank'] / df_merged.shape[0] )

In [237]:
df_merged['quality_index_time'] = df_merged['similarity_scaled'] * df_merged['bills_quantile']
df_merged['quality_index_sum'] = df_merged['similarity_scaled'] + df_merged['bills_quantile']

In [238]:
df_merged.sort_values(by=['similarity'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
0,金光琳 김광림 미래통합당,51.81,1.000000,0.074637,227,0.230508,0.230508,1.230508
1,崔運烈 최운열 더불어민주당,51.80,0.999377,0.122140,169,0.427119,0.426853,1.426496
2,金賢美 김현미 더불어민주당,51.57,0.985047,0.076087,225,0.237288,0.233740,1.222335
3,金太年 김태년 더불어민주당,51.34,0.970717,0.074045,231,0.216949,0.210596,1.187666
4,洪宜洛 홍의락 더불어민주당,50.94,0.945794,0.118421,174,0.410169,0.387936,1.355964
5,韓善敎 한선교 미래한국당,50.46,0.915888,0.043421,274,0.071186,0.065199,0.987074
6,金斗官 김두관 더불어민주당,50.39,0.911526,0.110902,182,0.383051,0.349161,1.294577
7,李貞鉉 이정현 무소속,50.38,0.910903,0.035088,284,0.037288,0.033966,0.948192
8,姜吉夫 강길부 무소속,50.36,0.909657,0.031618,285,0.033898,0.030836,0.943556
9,吳濟世 오제세 더불어민주당,50.29,0.905296,0.166301,123,0.583051,0.527834,1.488347


In [239]:
df_merged.sort_values(by=['quality_index_time'], ascending=False)

,name_party,similarity,similarity_scaled,bills_index_scaled,rank,bills_quantile,quality_index_time,quality_index_sum
28,朴釘 박정 더불어민주당,49.69,0.867913,0.651089,3,0.989831,0.859087,1.857743
66,李燦烈 이찬열 미래통합당,48.97,0.823053,1.000000,1,0.996610,0.820263,1.819663
35,魏聖坤 위성곤 더불어민주당,49.51,0.856698,0.425987,13,0.955932,0.818945,1.812630
53,尹厚德 윤후덕 더불어민주당,49.17,0.835514,0.463497,8,0.972881,0.812856,1.808395
85,金政祐 김정우 더불어민주당,48.78,0.811215,0.450998,9,0.969492,0.786466,1.780706
111,黃柱洪 황주홍 민생당,48.43,0.789408,0.833333,2,0.993220,0.784056,1.782628
38,金玄權 김현권 더불어민주당,49.49,0.855452,0.365351,25,0.915254,0.782956,1.770706
65,鄭仁和 정인화 무소속,48.99,0.824299,0.414474,15,0.949153,0.782386,1.773452
103,金哲玟 김철민 더불어민주당,48.56,0.797508,0.537281,6,0.979661,0.781287,1.777169
143,金鍾懷 김종회 민생당,48.17,0.773209,0.632723,4,0.986441,0.762725,1.759649


In [242]:
senators_list = ['박정 더불어민주당', '위성곤 민생당', '이찬열 민생당', '윤후덕 미래한국당', '김정우 정의당']
show_senators_bills_contents(senators_list, bills)

{'박정 더불어민주당': ['지역특화발전특구에 대한 규제특례법 전부개정법률안(정성호의원 등 10인)',
  '중소기업 인력지원 특별법 일부개정법률안(김경수의원 등 14인)',
  '공공기관 지방이전에 따른 혁신도시 건설 및 지원에 관한 특별법 일부개정법률안(이찬열의원 등 10인)'],
 '위성곤 민생당': ['청년고용촉진 특별법 일부개정법률안(어기구의원 등 14인)',
  '중소기업창업 지원법 일부개정법률안(박정의원 등 19인)',
  '지방공무원법 일부개정법률안(박정의원 등 10인)'],
 '이찬열 민생당': ['벤처기업육성에 관한 특별조치법 일부개정법률안(김수민의원 등 11인)',
  '소상공인 보호 및 지원에 관한 법률 일부개정법률안(홍의락의원 등 10인)',
  '청년세법안(정세균의원 등 108인)'],
 '윤후덕 미래한국당': ['국가재정법 일부개정법률안(정세균의원 등 108인)',
  '청년고용촉진 특별법 일부개정법률안(박정의원 등 16인)',
  '항공사업법 일부개정법률안(함진규의원 등 10인)'],
 '김정우 정의당': ['조달사업에 관한 법률 전부개정법률안(김정우의원 등 12인)',
  '공공보건의료에 관한 법률 일부개정법률안(원혜영의원 등 10인)',
  '독점규제 및 공정거래에 관한 법률 일부개정법률안(김병관의원 등 21인)']}